# Osnovi operativnih sistema
## ```Projektni zadatak - Andrej Tomić 1106/22```
## Bankarski algoritam

```Funkcija za učitavanje podataka iz ulaznog fajla```

In [1]:
def ucitavanje_ulaznog_fajla(naziv_fajla):
    with open(naziv_fajla, 'r') as file:
        linije = file.readlines()
    
    podaci = {}
    podaci['broj_procesa'] = int(linije[0].split(':')[1].strip())
    podaci['naziv_procesa'] = linije[1].split(':')[1].strip().split(' ')
    podaci['broj_resursa'] = int(linije[2].split(':')[1].strip())
    podaci['nazivi_resursa'] = linije[3].split(':')[1].strip().split(' ')
    podaci['broj_instanci_resursa'] = list(map(int, linije[4].split(':')[1].strip().split(' ')))
    podaci['vektor_raspolozivosti'] = list(map(int, linije[5].split(':')[1].strip().split(' ')))

    def ucitaj_matricu(pocetna_linija, broj_redova):
        matrica = []
        for i in range(pocetna_linija, pocetna_linija + broj_redova):
            matrica.append(list(map(int, linije[i].strip().split(' '))))
        return matrica
    
    podaci['matrica_alokacije'] = ucitaj_matricu(7, podaci['broj_procesa'])
    podaci['matrica_zahtjeva'] = ucitaj_matricu(7 + podaci['broj_procesa'] + 1, podaci['broj_procesa'])

    pozicija_liste_zahtjeva = 7 + 2 * podaci['broj_procesa'] + 2
    podaci['lista_zahtjeva'] = []
    for i in range(pozicija_liste_zahtjeva, len(linije)):
        podjela = linije[i].strip().split(' ')
        naziv_procesa = podjela[0]
        zahtjev = list(map(int, podjela[1:]))
        podaci['lista_zahtjeva'].append((naziv_procesa, zahtjev))

    return podaci

```Funkcija za računanje matrice potreba```

In [2]:
def matrica_potreba(matrica_zahtjeva, matrica_alokacije):
    potreba = []
    for i in range(len(matrica_zahtjeva)):
        potreba.append([matrica_zahtjeva[i][j] - matrica_alokacije[i][j] for j in range(len(matrica_zahtjeva[0]))])
    return potreba

```Implementacija bankarskog algoritma```

In [3]:
def bankarski_algoritam(vektor_raspolozivosti, matrica_alokacije, matrica_zahtjeva):
    broj_procesa = len(matrica_alokacije)
    broj_resursa = len(vektor_raspolozivosti)

    rad = vektor_raspolozivosti[:]
    zavrseni = [False] * broj_procesa
    sigurno = []

    while len(sigurno) < broj_procesa:
        pronadjen = False
        for i in range(broj_procesa):
            if not zavrseni[i] and all(matrica_zahtjeva[i][j] <= rad[j] for j in range(broj_resursa)):
                for j in range(broj_resursa):
                    rad[j] += matrica_alokacije[i][j]
                zavrseni[i] = True
                sigurno.append(i)
                pronadjen = True
                break
        if not pronadjen:
            return False, []
    
    return True, sigurno

```Provjera zahtjeva```

In [4]:
def odobren_zahtjev(indeks_procesa, lista_zahtjeva, vektor_raspolozivosti, matrica_alokacije, matrica_potreba):
    broj_resursa = len(vektor_raspolozivosti)
    
    if any(lista_zahtjeva[i] > matrica_potreba[indeks_procesa][i] for i in range(broj_resursa)):
        return False
    
    if any(lista_zahtjeva[i] > vektor_raspolozivosti[i] for i in range(broj_resursa)):
        return False
    
    vektor_raspolozivosti_kopija = vektor_raspolozivosti[:]
    matrica_alokacije_kopija = [red[:] for red in matrica_alokacije]
    matrica_potreba_kopija = [red[:] for red in matrica_potreba]

    for i in range(broj_resursa):
        vektor_raspolozivosti_kopija[i] -= lista_zahtjeva[i]
        matrica_alokacije_kopija[indeks_procesa][i] += lista_zahtjeva[i]
        matrica_potreba_kopija[indeks_procesa][i] -= lista_zahtjeva[i]

    sigurno, _ = bankarski_algoritam(vektor_raspolozivosti_kopija, matrica_alokacije_kopija, matrica_potreba_kopija)
    return sigurno

```Izvršavanje simulacije```

In [5]:
def simulacija(naziv_ulaznog_fajla):
    podaci = ucitavanje_ulaznog_fajla(naziv_ulaznog_fajla)
    matrica_potreba_ = matrica_potreba(podaci['matrica_zahtjeva'], podaci['matrica_alokacije'])
    
    print("--- Rezultati simulacije ---")
    print("Matrica potreba:")
    for red in matrica_potreba_:
        print("   " + ' '.join(map(str, red)))

    sigurno, sigurno_stanje_ = bankarski_algoritam(podaci['vektor_raspolozivosti'], podaci['matrica_alokacije'], matrica_potreba_)
    if sigurno:
        sekvenca_procesa = ' -> '.join(podaci['naziv_procesa'][i] for i in sigurno_stanje_)
        print(f"Sistem je u stabilnom stanju ({sekvenca_procesa}).")
    else:
        print("Sistem nije u stabilnom stanju.")

    for zahtjev in podaci['lista_zahtjeva']:
        naziv_procesa = zahtjev[0]
        indeks_procesa = podaci['naziv_procesa'].index(naziv_procesa)
        zahtjev_resursa = zahtjev[1]

        if odobren_zahtjev(indeks_procesa, zahtjev_resursa, podaci['vektor_raspolozivosti'], podaci['matrica_alokacije'], matrica_potreba_):
            print(f"Zahtjev ({naziv_procesa}, {', '.join(map(str, zahtjev_resursa))}) će biti odobren.")
        else:
            print(f"Zahtjev ({naziv_procesa}, {', '.join(map(str, zahtjev_resursa))}) neće biti odobren.")

# Pokretanje simulacije
print('Prva simulacija:')
simulacija('ulaz.txt')

print('\nDruga simulacija:')
simulacija('ulaz2.txt')

print('\nTreća simulacija:')
simulacija('ulaz3.txt')

Prva simulacija:
--- Rezultati simulacije ---
Matrica potreba:
   7 4 3
   1 2 2
   6 0 0
   0 1 1
   4 3 1
Sistem je u stabilnom stanju (P1 -> P3 -> P0 -> P2 -> P4).
Zahtjev (P1, 1, 1, 1) će biti odobren.
Zahtjev (P2, 3, 0, 0) će biti odobren.
Zahtjev (P4, 3, 3, 1) neće biti odobren.

Druga simulacija:
--- Rezultati simulacije ---
Matrica potreba:
   5
   2
   7
Sistem je u stabilnom stanju (P1 -> P0 -> P2).

Treća simulacija:
--- Rezultati simulacije ---
Matrica potreba:
   1 1 6
   4 4 1
   1 2 5
   1 3 1
   1 0 7
   2 0 6
Sistem je u stabilnom stanju (P4 -> P1 -> P2 -> P3 -> P5 -> P6).
Zahtjev (P1, 0, 1, 0) neće biti odobren.
Zahtjev (P2, 0, 0, 1) neće biti odobren.
Zahtjev (P3, 1, 0, 0) će biti odobren.
